In [50]:
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

In [51]:
file='../data/first.xlsx'
file2='../data/second.xlsx'

In [52]:
df=pd.read_excel(file)
df2=pd.read_excel(file2)
df= pd.concat([df, df2], axis=0)

In [53]:
# 인물 컬럼에서 "#"으로 시작하는 행은 제거하는 코드 
df = df[df["인물"].str.startswith("#") == False]

In [54]:
people=list(df["인물"].value_counts()[:50].index) # 대사가 많은 인물만 추리기 

In [55]:
# 두한, 청년두한, 소년두한
# 정진영, 청년진영 
# 개코, 소년개코 
# 해당 인물을 같은 라벨로 표시하기 : replace사용
replace_dict = {
    "청년두한": "두한",
    "소년두한": "두한",
    "청년진영": "정진영",
    "청년개코": "개코"
}

df["인물"] = df["인물"].replace(replace_dict)
df["인물"].value_counts()[:50]

인물
두한      5677
이정재     2158
김영태     1803
임화수     1492
정진영     1314
최동열     1141
개코      1040
김무옥      966
미와       943
문영철      861
시라소니     828
하야시      687
조병옥      659
곽영주      653
이기붕      618
신영균      611
유지광      594
이승만      585
김기홍      584
유진산      583
나레이션     536
이화룡      534
삼수       497
나미꼬      474
구마적      471
이석재      455
원노인      442
눈물       439
설향       342
김관철      341
친할머니     341
쌍칼       337
번개       335
정대발      333
애란       294
오숙근      293
김동진      289
김천호      284
이억일      276
정팔       272
한백수      272
장택상      268
이영숙      266
소년개코     265
왕발       258
마루오까     253
조열승      228
박인애      227
오무라      227
와싱턴      225
Name: count, dtype: int64

In [56]:
import re

def remove_punctuations(text):
    return re.sub("[.,:;?!*~]", "", text)

df["대사"] = df["대사"].apply(remove_punctuations)

In [57]:
people=['김두한',
    '쌍칼',
    '문영철',
    '김무옥',
    '번개',
    '김영태',
    '털보',
    '삼수',
    '병수',
    '와싱턴',
    '신영균',
    '개코',
    '홍만길',
    '휘발유',
    '정진영',
    '김관철',
    '갈치',
    '아구',
    '홍영철',
    '조일환',
        '시라소니',
        '이정재',
        '구마적',
        '임화수',
        '이석재',
        '이화룡',
        '정팔']
df=df[df["인물"].isin(people)].reset_index(drop=True)
df.to_csv('../data/우미관 패거리.csv', index=False)

In [58]:
# '인물' 값과 서열을 매핑한 딕셔너리 생성
# 오야붕급>= 준오야급>= 오야보좌급>= 상급전투원급>= 중급전투원급>=하급전투원급>=말단급>=닭대가리급>=피라미급
power_mapping = {
    '김두한':"오야붕급" ,
    '쌍칼': "오야붕급",
    '문영철':'오야보좌급',
    '김무옥':'오야보좌급' ,
    '번개':'하급전투원급' ,
    '김영태':'준오야급' ,
    '털보':'하급전투원급' ,
    '삼수':'하급전투원급' ,
    '병수':'피라미급' ,
    '와싱턴':'하급전투원급' ,
    '신영균':'상급전투원급' ,
    '개코':'하급전투원급' ,
    '홍만길':'중급전투원급' ,
    '휘발유':'하급전투원급' ,
    '정진영':'상급전투원급' ,
    '김관철':'말단급' ,
    '갈치':'피라미급' ,
    '아구':'피라미급' ,
    '홍영철':'닭대가리급' ,
    '조일환':'닭대가리급',
    '시라소니':'오야붕급',
    '이정재':'오야붕급',
    '구마적':'오야붕급',
    '임화수':'중급전투원급',
    '이석재':'중급전투원급',
    '이화룡':'오야붕급',
    '정팔':'준오야급'
}

# '전투력' 열 추가 및 전투력 값 매핑
df['서열'] = df['인물'].map(power_mapping)

In [59]:
label = df["서열"]
feature = df["대사"]

[라벨 인코딩]

In [60]:
# label 을 인코딩하기
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
encoder.fit(label)
label = encoder.transform(label)
df["서열"]=label
df["서열"].unique()

array([4, 8, 2, 3, 5, 7, 6, 1, 0])

In [61]:
encoder.classes_

array(['닭대가리급', '말단급', '상급전투원급', '오야보좌급', '오야붕급', '준오야급', '중급전투원급',
       '피라미급', '하급전투원급'], dtype=object)

In [62]:
df

,회차,시간,인물,대사,서열
0,1,0~5,김두한,가마에 불을 지피신다구요,4
1,1,0~5,김두한,열심히 하시는 모습이 아주 좋아 보이십니다,4
2,1,0~5,김두한,한동안 이 제 별장이 비어있었습니다,4
3,1,0~5,김두한,더는 나랏일하기가 어렵게 됐습니다 군사독잰 계속되고 민주주의는 멀었습니다 무식한 이...,4
4,1,0~5,김두한,이 물러나기는 해야겠는데 그냥 그만 둔다는 것도 체면이 안 서고 해서…,4
...,...,...,...,...,...
15436,124,50~55,조일환,명심하고 있습니다 늘 큰형님의 가르침을 가슴에 품고 살고 있습니다,0
15437,124,55~60,조일환,예 큰형님 영원히 그렇게 살겠습니다,0
15438,124,55~60,조일환,예 큰형님,0
15439,124,55~60,조일환,괜찮으시겠습니까,0


In [63]:
# from sklearn.preprocessing import LabelEncoder
# encoder = LabelEncoder()
# encoder.fit(label)
# label = encoder.transform(label)
# df["전투력"]=label
# df["전투력"].unique()

In [64]:
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
    def __init__(self, corpus):
        nsmcDF = pd.DataFrame(corpus).fillna('')

        x_data = nsmcDF['대사'].values
        self.x_data = x_data
        self.y_data = nsmcDF['서열'].values

    def __len__(self):
        return len(self.x_data)

    def __getitem__(self, idx):
        x = self.x_data[idx]
        y = self.y_data[idx]
        return y, x

In [65]:
trainDS = CustomDataset(df)

In [66]:
### ===> 모듈 로딩
from konlpy.tag import Komoran
from torchtext.vocab import build_vocab_from_iterator

In [67]:
### ===> 토큰관련 특별 문자
UNK = '<UNK>'
PAD = '<PAD>'

In [68]:
### 토큰화 인스턴스 생성
tokenizer = Komoran()

In [69]:
def yield_tokens(data_iter):
    for label, text in data_iter:
        yield tokenizer.morphs(text)

In [70]:
### ===> 토큰화 및 단어/어휘 사전 생성
VOCAB = build_vocab_from_iterator(
    yield_tokens(trainDS),
    min_freq=2,
    specials= [PAD, UNK],
    special_first=True
)

### <UNK> 인덱스 설정
VOCAB.set_default_index(VOCAB[UNK])

In [71]:
trainDS = CustomDataset(df)

In [72]:
### ===> 모듈 로딩
from konlpy.tag import Komoran
from torchtext.vocab import build_vocab_from_iterator

In [73]:
### ===> 토큰관련 특별 문자
UNK = '<UNK>'
PAD = '<PAD>'

In [74]:
### 토큰화 인스턴스 생성
tokenizer = Komoran()

In [75]:
def yield_tokens(data_iter):
    for label, text in data_iter:
        yield tokenizer.morphs(text)

In [76]:
### ===> 토큰화 및 단어/어휘 사전 생성
VOCAB = build_vocab_from_iterator(
    yield_tokens(trainDS),
    min_freq=2,
    specials= [PAD, UNK],
    special_first=True
)

### <UNK> 인덱스 설정
VOCAB.set_default_index(VOCAB[UNK])

In [77]:
### ===> 텍스트 >>>> 정수 인코딩
text_pipeline = lambda x: VOCAB(tokenizer.morphs(x))

### ===> 레이틀 >>> 정수 인코딩 (0~3)
label_pipeline = lambda x: int(x)

In [78]:
### 인코딩 : 문자 >>>> 숫자로 변환
token_to_id ={ label : id  for label, id in VOCAB.get_stoi().items()}

### 디코딩 : 숫자 >>>> 문자로 변환
id_to_token ={ id : label  for label, id in VOCAB.get_stoi().items()}


In [79]:
### ===> 모듈로딩
from torch.utils.data import DataLoader
from torch.utils.data.dataset import random_split
from torch.nn.utils.rnn import pad_sequence
import torch 

In [80]:
### ===> 실행 디바이스 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [81]:
# 배치크기만큼 데이터셋 반환 함수
def collate_batch(batch):
    # 배치크기 만큼의 라벨, 텍스트, 오프셋 값 저장 변수
    label_list, text_list, offsets = [], [], [0]
    
    # 1개씩 뉴스기사, 라벨 추출해서 저장
    for (_label, _text) in batch:
        # 라벨 인코딩 후 저장
        label_list.append(label_pipeline(_label))
        
        # 텍스트 인코딩 후 저장
        processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
        text_list.append(processed_text)
        
        # 텍스트 offset 즉, 텍스트 크기/길이 저장 
        offsets.append(processed_text.size(0))
    
    # 텐서화 진행 
    label_list = torch.tensor(label_list, dtype=torch.int64)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text_list = torch.cat(text_list)
    
    return label_list.to(device), text_list.to(device), offsets.to(device)

In [82]:
### ===> 학습용, 검증용, 테스트용 DataSet 준비 
BATCH_SIZE = 32

### 학습용, 검증용, 테스트용 Dataset, DataLoader 준비
num_train = int(len(trainDS) * 0.95)
print(f' num_train :{num_train}')

split_trainDS, split_validDS= random_split( trainDS, [num_train, len(trainDS) - num_train])
print(f' len(split_trainDS) :{len(split_trainDS)}')
print(f' len(split_validDS) :{len(split_validDS)}')

trainDL = DataLoader( split_trainDS, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch )
validDL = DataLoader( split_validDS, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch )

 num_train :14668
 len(split_trainDS) :14668
 len(split_validDS) :773


In [83]:
trainDS

In [84]:
print(f' len(trainDL) :{len(trainDL)*BATCH_SIZE}')
print(f' len(validDL) :{len(validDL)*BATCH_SIZE}')

 len(trainDL) :14688
 len(validDL) :800


In [85]:
for i in trainDL:
    print(i)
    break

(tensor([3, 6, 8, 4, 3, 8, 1, 2, 3, 4, 8, 8, 4, 4, 5, 4, 4, 8, 3, 5, 6, 6, 6, 6,
        1, 5, 5, 4, 6, 5, 3, 4]), tensor([  36,   27,    5,   44,   29,   12,   30,   55,  753,  282,    2,  716,
          62,   13,  172,   30,  134,   13,    6,   23,  933,    4,  328,   63,
          15,   26,    6,  174,  933,   22,   50,  450,   77,   30,   22,   30,
          22,   30,   22,    1,  933,  403,   30,  508,  229,  121,   57,    3,
         236, 4425,   78,  504, 2186,   31,   11,   17,   98, 1043,  229,  177,
           5,   46,   18,  116,  329,   77,  755,  433,    5,  632,   30,   29,
          16,   24,   54,   25,    7,  171,   14,    3,    5,  768,    8,  160,
          54,   34,  116,  286,   48,  388,  323,    3,   32,   38,   34,   17,
         392, 2056, 1003,   11,  160,   24,  187,  232,  403,  287,    3,    8,
          84,   94,   77,   99,   22,    4,   60,   17,  603,  593,  286, 2726,
         329,  111,    6, 5223,  214,    8,   20,   30,  322,   30,   34,   22,
     

In [86]:
import torch.nn as nn

# 은닉층 : Linear - 4개 클래스 분류 
class TextModel(nn.Module):
    def __init__(self, VOCAB_SIZE, EMBEDD_DIM, HIDDEN_SIZE, NUM_CLASS):
        super(TextModel, self).__init__()
        # 모델 구성 층 정의 
        self.embedding = nn.EmbeddingBag(VOCAB_SIZE, EMBEDD_DIM, sparse=False)
        self.fc = nn.Linear(EMBEDD_DIM, NUM_CLASS)
        self.init_weights()
    
    # 가중치 초기화
    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()
    
    # 순방향 학습 진행
    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        return self.fc(embedded)

In [87]:
# 학습 관련 파라미터와 인스턴스 
HIDDEN_SIZE=3
EMBEDD_DIM=64
VOCAB_SIZE = len(VOCAB)
NUM_CLASS = 9
EPOCHS = 20
LR = 5
BATCH_SIZE = 64

In [88]:
# 학습 관련 인스턴스
import torch.optim as optim
MODEL = TextModel(VOCAB_SIZE, EMBEDD_DIM, HIDDEN_SIZE, NUM_CLASS).to(device)
CRITERION = nn.CrossEntropyLoss()
OPTIMIZER = optim.AdamW(MODEL.parameters(), lr=LR)
SCHEDULER = optim.lr_scheduler.StepLR(OPTIMIZER, 1.0, gamma=0.1) # learning rate를 줄이는 용도 

In [89]:

# 학습 관련 함수 정의 


def train(model, dataloader, optimizer, criterion, epoch):
    model.train()

    # 학습 평가 관련 변수들
    total_acc, total_count = 0, 0
    log_interval = 300

    for idx, (label, text, offsets) in enumerate(dataloader):
        predicted_label = model(text, offsets)
        optimizer.zero_grad()
        loss = criterion(predicted_label, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optimizer.step()

        # 배치 학습 평가
        total_acc += (predicted_label.argmax(1) == label).sum().item()
        total_count += label.size(0)

        if idx % log_interval == 0 and idx > 0:
            print(f"epoch : {epoch} batch : {idx} loss : {loss.item()}")
            print(f"Train_Accuracy : {total_acc / total_count}")
            total_acc, total_count = 0, 0

            break


In [90]:
def evaluate(model, dataloader, criterion):
    model.eval()
    
    total_acc, total_count = 0,0
    
    with torch.no_grad():
        for idx, (label, text, offsets) in enumerate(dataloader):
            predicted_label = model(text, offsets)
            loss = criterion(predicted_label, label)
            total_acc += (predicted_label.argmax(1) == label).sum().item()
            total_count += label.size(0)
            
    return total_acc/total_count

In [91]:
def predict(model, text, text_pipeline):
    with torch.no_grad():
        text = torch.tensor(text_pipeline(text), dtype=torch.int64).to(device)
        # text = text.unsqueeze(0)
        offsets = torch.tensor([0]).to(device)
        predicted_label = model(text, offsets)
        return predicted_label.argmax(1).item() + 1

In [92]:
# 학습 진행
for epoch in range(1, EPOCHS+1):
    train(MODEL, trainDL, OPTIMIZER, CRITERION, epoch)
    accu_val = evaluate(MODEL, validDL, CRITERION)
    print(f"epoch : {epoch} Valid_Accuracy : {accu_val}")
    SCHEDULER.step()

epoch : 1 batch : 300 loss : 194.88204956054688
Train_Accuracy : 0.2288205980066445
epoch : 1 Valid_Accuracy : 0.2794307891332471
epoch : 2 batch : 300 loss : 9.729482650756836
Train_Accuracy : 0.3155107973421927
epoch : 2 Valid_Accuracy : 0.33247089262613194
epoch : 3 batch : 300 loss : 1.9843506813049316
Train_Accuracy : 0.4804817275747508
epoch : 3 Valid_Accuracy : 0.4579560155239327
epoch : 4 batch : 300 loss : 0.9559847712516785
Train_Accuracy : 0.606312292358804
epoch : 4 Valid_Accuracy : 0.5148771021992238
epoch : 5 batch : 300 loss : 0.9879356026649475
Train_Accuracy : 0.6345514950166113
epoch : 5 Valid_Accuracy : 0.51875808538163
epoch : 6 batch : 300 loss : 1.1832979917526245
Train_Accuracy : 0.6414036544850499
epoch : 6 Valid_Accuracy : 0.51875808538163
epoch : 7 batch : 300 loss : 1.2904080152511597
Train_Accuracy : 0.6401578073089701
epoch : 7 Valid_Accuracy : 0.51875808538163
epoch : 8 batch : 300 loss : 0.9217106699943542
Train_Accuracy : 0.647217607973422
epoch : 8 Vali

[모델 저장]

In [93]:
torch.save(MODEL, "YAINCLASSIFICATION.pth")

In [115]:


# 모델 로드
# model = TextModel(VOCAB_SIZE, EMBEDD_DIM, HIDDEN_SIZE, NUM_CLASS)
model = torch.load("YAINCLASSIFICATION.pth")
model.eval()
real_class = encoder.classes_

# 시연
text = "여러분 안심해주십시오. 서울을 끝까지 사수하겠습니다."
predicted_label = predict(model, text, text_pipeline)
print(predicted_label)
if predicted_label==9:
    predicted_label-=1
print(f"예측된 라벨: {real_class[predicted_label]}")

1
예측된 라벨: 말단급
